Notebook to evaluate and then illustrate the performance improvement of training on the psuedo synthetic catalog

In [ ]:
#!/usr/bin/env python3

from itertools import product
from collections import deque

import os
import pandas as pd
import glob
import numpy as np

import datetime
from scipy import signal
import pywt

import pyarrow as pa
import pyarrow.parquet as pq
import json

import joblib

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import os
import sys
sys.path.insert(0, os.path.abspath('../../bin/models'))

from nested_xval_utils import *

# Strategy:

Run nested cross validation on unavco gnss dataset.
Limit unavco dataset to comparable events (<70km radius)

for each run, train a model on just gnss data, and then compare the results with the complete model from the NGA dataset
Report Pr, Recall, F1 for these 10 runs.

Then train an overall model on the GNSS data.
Compare with teh overall NGA model against an unseen ambient dataset.

In [ ]:
fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = fsize-5
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

In [ ]:
#NGA SYNTHETIC DATA
##############
pq_list=os.listdir('../../data/feature_sets/')
pq_list=[os.path.join('../../data/feature_sets/',f) for f in os.listdir('../../data/feature_sets/')]
meta_list=[read_meta(pq_fs) for pq_fs in pq_list if ".pq" in pq_fs]
meta_df=pd.DataFrame.from_records(meta_list)

######################
ambient_list= list(meta_df[meta_df.magnitude.isnull()].eq_name.unique())
event_list=meta_df[~meta_df.magnitude.isnull()].sort_values(['magnitude'], ascending=False).groupby("eq_name").count().sort_values(['station'], ascending=False).index.tolist()
full_list=ambient_list+event_list

#convert to rsn
full_list_nga=meta_df[meta_df.eq_name.isin(full_list)].record_number.unique()
event_list_nga=meta_df[meta_df.eq_name.isin(event_list)].record_number.unique()

## JGR GNSS DATA
###############  Generate list of samples by event
pq_list=[os.path.join('../../data/jgr_data/feature_set/',f) for f in os.listdir('../../data/jgr_data/feature_set/')]

meta_list=[read_meta(pq_fs) for pq_fs in pq_list if ".pq" in pq_fs]
meta_df=pd.DataFrame.from_records(meta_list)

#jgr_test=meta_df[(meta_df.radius_from_event<70) | (meta_df.radius_from_event==np.nan)]
jgr_test=meta_df[(meta_df.radius_from_event<70) | (meta_df['radius_from_event'].isna())]

### dataframes

* jgr_test: all jgr real 5hz event featuresets plus ambient datasets
* full_list_nga: all waveforms in nga list + ambient datasets
* event_list_nga: all waveforms in nga list

In [ ]:
fs={'feature':['psd_t'], 'stacking':['horizontal'], 'dims':[['H0','H1','UP']], 'augment':[True]}
feature_sets=[dict(zip(fs, v)) for v in product(*fs.values())]

d = {'n_folds':[5],'max_depth': [50], 'n_estimators': [120], 'class_wt':[None, 'balanced'],'wl_thresh':[-30]}
hyperp=[dict(zip(d, v)) for v in product(*d.values())]

params=[i | feature_sets[0] for i in hyperp]

In [ ]:
X_train_nga, y_train_nga, name_list_nga, times_nga, snr_metric=list_to_featurearrays(event_list_nga, params[0], test=False)

In [ ]:
#ordered event list to roughly distribute testing 
ambient_list= list(meta_df[meta_df.magnitude.isnull()].eventID.unique())
event_list=jgr_test[~jgr_test.magnitude.isnull()].sort_values(['magnitude'], ascending=False).groupby("eventID").count().sort_values(['station'], ascending=False).index.tolist()
full_list=event_list#+ambient_list

In [ ]:
full_list_df=jgr_test[jgr_test.eventID.isin(full_list)]

#X_train_nga, y_train_nga, name_list_nga, times_nga, snr_metric=list_to_featurearrays(full_list_df, params[0], test=False)

In [ ]:
#for k in np.arange(num_runs):
num_runs=10
outer_results=[]

y_nga_pred=[]
y_nga_test=[]

y_jgr_pred=[]
y_jgr_test=[]
for k in np.arange(10):
    run=k+1
    items = deque(full_list)
    items.rotate(-k)
    test_set=list(items)[::num_runs]
    train_set=list(set(full_list) - set(test_set))
    
    test_set_df=jgr_test[jgr_test.eventID.isin(test_set)]
    train_set_df=jgr_test[jgr_test.eventID.isin(train_set)]
    
    #for features in feature_sets:
    for use_nga in [False,True]:

        if not use_nga:
            best_est_, stats=grid_search_jgr(train_set, params, X_train_nga, y_train_nga, jgr_test, include_nga=use_nga)
            threshold=stats.threshold # Hyper Param from xval training

            X_train, y_train, name_list, times, snr_metric=list_to_featurearrays_JGR(train_set_df, best_est_, test=True)

            clf = RandomForestClassifier(n_estimators=best_est_['n_estimators'], max_depth=best_est_['max_depth'], class_weight=best_est_['class_wt'],random_state=10, n_jobs=-1).fit(X_train, y_train)
        if use_nga:
            threshold=.43#.38
            clf = joblib.load('../../models/synth_model_all_%s.pkl' %int(threshold*100))

        X_test, y_test, name_list, times, snr_metric=list_to_featurearrays_JGR(test_set_df, params[0], test=True)
        y_pred_prob=clf.predict_proba(X_test)[:, 1]

        y_pred = (y_pred_prob >= threshold).astype('int')
        ###
        # evaluate the model on test data
        p, r, f1, blah=precision_recall_fscore_support(y_test, y_pred, average='binary')
        print(p,r,f1)

        from sklearn.metrics import precision_recall_curve
        precisions, recalls, thresholds = precision_recall_curve(y_test, y_pred_prob)

        # store the result
        outer_results.append([p,r,f1,threshold, precisions, recalls, thresholds, y_test, y_pred_prob, best_est_, run, best_est_['feature'], test_set, use_nga])
        # report progress
        print('>f1=%.3f' % (f1)) 
        
        if use_nga:
            y_nga_pred.append(y_pred)
            y_nga_test.append(y_test)
        else:
            y_jgr_pred.append(y_pred)
            y_jgr_test.append(y_test)

        #executionTime = (time.time() - startTime)
        #print('Execution time in seconds: ' + str(executionTime))

In [ ]:
results_df=pd.DataFrame(outer_results, 
                        columns=['precision','recall','f1','threshold', 
                                 'precisions', 'recalls', 'thresholds', 
                                 'y_test', 'y_pred_prob', 'best_est_', 'run', 
                                 'best_est_[feature]', 'test_set', 'use_nga'])

In [ ]:
res_df=[]
for title,y_test, y_pred in zip(['nga','jgr'],[y_nga_test,y_jgr_test],[y_nga_pred, y_jgr_pred]):
    y_test=np.concatenate( y_test, axis=0 )
    y_pred=np.concatenate( y_pred, axis=0 )
    p, r, f1, blah=precision_recall_fscore_support(y_test, y_pred, average='binary')
    print(title,p, r, f1)
    res_df.append([title,p, r, f1])

In [ ]:
res_df=pd.DataFrame(res_df, columns=['type','Precision','Recall','F1'])

In [ ]:
res_df.to_parquet('../../data/results/jgr_v_nga_table.pq')

In [ ]:
#To further evaluate in the future
res_df=pd.read_parquet('../../data/results/jgr_v_nga_table.pq')
X_train_nga=y_train_nga=np.nan

In [ ]:
# make overall model of JGR
full_set=jgr_test[jgr_test.eventID.isin(full_list)]

best_est_, stats=grid_search_jgr(full_list, params, X_train_nga, y_train_nga, jgr_test, include_nga=False)
threshold=stats.threshold # Hyper Param from xval training
#best_est_=params[0]
#stats.threshold=0.4

X_train, y_train, name_list, times, snr_metric=list_to_featurearrays_JGR(full_set, best_est_, test=True)

clf = RandomForestClassifier(n_estimators=best_est_['n_estimators'], max_depth=best_est_['max_depth'], class_weight=best_est_['class_wt'],random_state=10, n_jobs=-1).fit(X_train, y_train)
keep_thresh=str(int(100*stats.threshold))
    
joblib.dump(clf, '../../models/complete_JGR_model_%s.pkl' %keep_thresh)

In [ ]:
full_set=jgr_test[jgr_test.eventID.isin(full_list)]
X_train, y_train, name_list, times, snr_metric=list_to_featurearrays_JGR(full_set, best_est_, test=True)

In [ ]:
# Load in Ambient 2 set data
# test each model - TNR

In [ ]:
param=params[0]
X_, y_, name_list, times, snr_metric=list_to_featurearrays_ambient_test(param, test=True)


In [ ]:
tnr=[]
#for ds,clf_n in zip(['nga','jgr'],['synth_model_all_39.pkl','complete_JGR_model_40.pkl']):j
for ds,clf_n in zip(['nga','jgr'],['synth_model_all_43.pkl','complete_JGR_model_41.pkl']):
    threshold=0.4
    clf = joblib.load('../../models/'+clf_n)
    y_pred_prob=clf.predict_proba(X_)[:, 1]
    y_pred = (y_pred_prob >= threshold).astype('int')
    
    CM = confusion_matrix(y_, y_pred)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    FPR = FP/(FP+TN)
   # Specificity or true negative rate
    TNR = TN/(TN+FP)
    
    print(FPR,TNR)
    tnr.append(TNR)


In [ ]:
res_df['TNR']=tnr

In [ ]:
x_labels=['Precision','Recall','F1','Ambient TNR']
colors=['#1b9e77','#7570b3']
ind = np.arange(len(x_labels))  # the x locations for the groups
width = 0.3       # the width of the bars

fig = plt.figure(figsize=(8,3))
ax = fig.add_subplot(111)
rects=[]
for i,(feature,label,color) in enumerate(zip(['nga','jgr'],['Trained on Augmented SM Catalog','Trained on GNSS Catalog'],colors)):
    rects1=ax.bar(ind+width*i, res_df[res_df.type==feature][['Precision','Recall','F1','TNR']].values[0], width, label=label, color=color)
    
ax.set_ylim([0.65,1.01])
ax.set_ylabel('Score')
ax.set_xticks(ind+width)
ax.set_xticklabels( x_labels )

ax.text(ind[0], float(res_df[res_df.type=='nga']['Precision']),'+{0:.2f}'.format(-res_df.Precision.diff()[1]))
ax.text(ind[1], float(res_df[res_df.type=='nga']['Recall']),'+{0:.2f}'.format(-res_df.Recall.diff()[1])) #+width/1.9
ax.text(ind[2], float(res_df[res_df.type=='nga']['F1']),'+{0:.2f}'.format(-res_df.F1.diff()[1]))
#ax.text(ind[3], float(res_df[res_df.type=='nga']['TNR']),'{0:.2f}'.format(res_df.TNR.diff()[1]))

#ax.legend( (rects1[0], rects1[1]), ['nga','jgr'] , ncol=4, title='Features', loc='upper left')
ax.legend()
#plt.savefig('figs/why_train_on_sm.png',dpi=300, bbox_inches='tight')

In [ ]:
fsize = 20
tsize = 20
tdir = 'in'
major = 5.0
minor = 3.0
lwidth = 0.8
lhandle = 2.0
plt.style.use('default')
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = fsize-3
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['axes.linewidth'] = lwidth
plt.rcParams['legend.handlelength'] = lhandle

In [ ]:
x_labels=['Precision','Recall','F1','Ambient TNR']
colors=['#1b9e77','#7570b3']
ind = np.arange(len(x_labels))  # the x locations for the groups
width = 0.3       # the width of the bars

fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)
rects=[]
for i,(feature,label,color) in enumerate(zip(['nga','jgr'],['Trained on Augmented SM Catalog','Trained on GNSS Catalog'],colors)):
    rects1=ax.bar(ind+width*i, res_df[res_df.type==feature][['Precision','Recall','F1','TNR']].values[0], width, label=label, color=color)
    
ax.set_ylim([0.65,1.01])
ax.set_ylabel('Score')
ax.set_xticks(ind+width)
ax.set_xticklabels( x_labels )

ax.text(ind[0], float(res_df[res_df.type=='nga']['Precision']),'+{0:.2f}'.format(-res_df.Precision.diff()[1]))
ax.text(ind[1], float(res_df[res_df.type=='nga']['Recall']),'+{0:.2f}'.format(-res_df.Recall.diff()[1])) #+width/1.9
ax.text(ind[2], float(res_df[res_df.type=='nga']['F1']),'+{0:.2f}'.format(-res_df.F1.diff()[1]))
#ax.text(ind[3], float(res_df[res_df.type=='nga']['TNR']),'{0:.2f}'.format(res_df.TNR.diff()[1]))

#ax.legend( (rects1[0], rects1[1]), ['nga','jgr'] , ncol=4, title='Features', loc='upper left')
ax.legend()
#plt.savefig('figs/why_train_on_sm.png',dpi=300, bbox_inches='tight')